In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import SequenceGen,CNN_RNN2seq,loss_func_generator
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds.npy')
means = np.load('../Data/means.npy')
long_normal_value = np.load('../Data/long_normal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = 5

In [5]:
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

In [6]:
val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

In [7]:
conv = Sequential(ConvBatchRelu1D(3,8,3),ConvBatchRelu1D(8,16,3))

In [8]:
rnn = GRU(16,32,2,batch_first=True)

In [9]:
linear = Sequential(Linear(32,32),ReLU(True),Linear(32,1))

In [10]:
model = CNN_RNN2seq(conv,rnn,linear).to('cuda:0')

In [11]:
#loss_func = loss_func_generator(MSELoss())
loss_func = loss_func_generator(L1Loss())

In [12]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [13]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:516.2224500863663, val_loss:423.1395568847656
epoch:1, train_loss:522.4179334768977, val_loss:419.86614990234375
epoch:2, train_loss:459.125440109332, val_loss:410.1759033203125
epoch:3, train_loss:484.43135341189685, val_loss:431.9317321777344
epoch:4, train_loss:474.0166014241431, val_loss:413.0675964355469
Training completed in 122.41518807411194s


In [ ]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:476.0807703838447, val_loss:458.9748840332031
epoch:1, train_loss:467.7023991046174, val_loss:416.1054992675781
epoch:2, train_loss:470.24124307187725, val_loss:416.5970153808594


In [57]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:453.40325307895483, val_loss:388.83056640625
epoch:1, train_loss:460.2913464798211, val_loss:399.4738464355469
epoch:2, train_loss:447.76573085488434, val_loss:388.8795166015625
epoch:3, train_loss:459.36252158762875, val_loss:417.07769775390625
epoch:4, train_loss:472.74120201150373, val_loss:411.1145324707031
Training completed in 116.52871489524841s


In [44]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:243731281.47686204, val_loss:352947872.0
epoch:1, train_loss:162734201.00873542, val_loss:384095360.0
epoch:2, train_loss:237149216.76161352, val_loss:402067968.0
epoch:3, train_loss:309695295.2885687, val_loss:395591584.0
epoch:4, train_loss:450304753.7159488, val_loss:371910048.0
Training completed in 115.90913200378418s


In [45]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:176948029.24694785, val_loss:426597440.0
epoch:1, train_loss:218645887.9099984, val_loss:275012864.0
epoch:2, train_loss:185636824.397636, val_loss:373682208.0
epoch:3, train_loss:247238947.61224094, val_loss:452415584.0
epoch:4, train_loss:154909668.08329016, val_loss:330578016.0
Training completed in 115.73838496208191s


In [ ]:
clip_grad_value_

In [56]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl,clip=0):
    # assume loss_func returns mean rather than sum
    since = time.time()
    opt.zero_grad()
    train_batch = len(train_dl.dataset)//train_dl.batch_size
    val_batch = len(valid_dl.dataset)//valid_dl.batch_size
    if clip!=0:
        paras = trainable_parameter(model)
        
    for epoch in range(epochs):
        # training #
        model.train()
        train_loss = 0
        for data in train_dl:
            loss = loss_func(model,data2cuda(data))
            loss.backward()
            if clip!=0:
                clip_grad_value_(paras,clip)
            opt.step()
            opt.zero_grad()
            train_loss += loss.item()
        
        # evaluating #
        val_loss = evaluate(model,valid_dl,loss_func,val_batch)
        print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/train_batch,val_loss))
    
    time_elapsed = time.time() - since
    print('Training completed in {}s'.format(time_elapsed))
    return model